<a href="https://colab.research.google.com/github/hari37200/data2detect-/blob/main/YOLO_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os

base_path = '/content/drive/MyDrive/project detect'
image_dir = os.path.join(base_path, 'TsignDet Test Database', 'test_image')
label_dir = os.path.join(base_path, 'TsignDet Test Database Annotation', 'lable')
converted_label_dir = os.path.join(base_path, 'converted_labels')

os.makedirs(converted_label_dir, exist_ok=True)


In [4]:
def convert_label(txt_path, img_path, save_path):
    with open(txt_path, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]

    if not os.path.exists(img_path):
        print(f"⚠️ Skipping: {img_path} not found or unreadable")
        return

    try:
        img = Image.open(img_path)
        w, h = img.size
    except:
        print(f"⚠️ Skipping unreadable image: {img_path}")
        return

    yolo_lines = []

    for i in range(0, len(lines)-1, 2):
        try:
            coords = list(map(int, lines[i].split(',')))
            cls = int(lines[i+1])
            x_center, y_center, width, height = convert_to_yolo_coords(coords, w, h)
            yolo_lines.append(f"{cls} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
        except Exception as e:
            print(f"⚠️ Skipping malformed pair in {txt_path}: {e}")
            continue

    if yolo_lines:
        with open(save_path, 'w') as f:
            f.write('\n'.join(yolo_lines))


In [5]:
import shutil
import random

dataset_path = os.path.join(base_path, 'dataset_yolo')
train_img_dir = os.path.join(dataset_path, 'images', 'train')
val_img_dir = os.path.join(dataset_path, 'images', 'val')
train_lbl_dir = os.path.join(dataset_path, 'labels', 'train')
val_lbl_dir = os.path.join(dataset_path, 'labels', 'val')

for d in [train_img_dir, val_img_dir, train_lbl_dir, val_lbl_dir]:
    os.makedirs(d, exist_ok=True)

all_labels = [f for f in os.listdir(converted_label_dir) if f.endswith('.txt')]
random.shuffle(all_labels)

split = int(0.8 * len(all_labels))
train_labels = all_labels[:split]
val_labels = all_labels[split:]

def move_files(label_list, img_dst, lbl_dst):
    for lbl_file in label_list:
        img_file = lbl_file.replace('.txt', '.bmp')
        src_img_path = os.path.join(image_dir, img_file)
        src_lbl_path = os.path.join(converted_label_dir, lbl_file)

        if not os.path.exists(src_img_path):
            print(f"⚠️ Missing image: {img_file}")
            continue

        shutil.copy(src_img_path, os.path.join(img_dst, img_file))
        shutil.copy(src_lbl_path, os.path.join(lbl_dst, lbl_file))

move_files(train_labels, train_img_dir, train_lbl_dir)
move_files(val_labels, val_img_dir, val_lbl_dir)


In [6]:
yaml_path = os.path.join(base_path, 'tsrd.yaml')
class_names = [f"sign_{i}" for i in range(59)]  # Adjust this if needed

with open(yaml_path, 'w') as f:
    f.write(f"path: {dataset_path}\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write("nc: 59\n")
    f.write(f"names: {class_names}\n")


In [ ]:
!pip install ultralytics --quiet

from ultralytics import YOLO

model = YOLO('yolov8s.pt')  # Or yolov8n.pt for smaller model
model.train(data=yaml_path, epochs=50, imgsz=640, batch=16, name='tsrd_yolov8')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralyti

100%|██████████| 21.5M/21.5M [00:00<00:00, 158MB/s]


Ultralytics 8.3.161 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/project detect/tsrd.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=tsrd_yolov8, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 

100%|██████████| 755k/755k [00:00<00:00, 14.7MB/s]

Overriding model.yaml nc=80 with nc=59

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      12/50         0G     0.7581     0.5545     0.8746         14        640: 100%|██████████| 26/26 [15:23<00:00, 35.53s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [01:00<00:00, 15.19s/it]

                   all        102        199      0.955       0.97      0.986      0.769



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      13/50         0G     0.7338     0.5385     0.8839         56        640:  58%|█████▊    | 15/26 [08:53<06:28, 35.28s/it]

In [1]:
!cp /content/runs/detect/tsrd_yolov8/weights/last.pt /content/drive/MyDrive/tsrd_last.pt
!cp /content/runs/detect/tsrd_yolov8/weights/best.pt /content/drive/MyDrive/tsrd_best.pt
